In [ ]:
from skimage import io, feature, color, exposure
from torch import nn
import pandas as pd
import numpy as np
import torch
import cv2

from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

In [ ]:
import matplotlib.pyplot as plt
import csv
import os
import tqdm

In [ ]:
data_split_dic = {i:{"train_range":(2*i*50,(2*i+1)*50-1),"test_range":((2*i+1)*50,(2*i+2)*50-1)} for i in range(20)}

def get_images(type="train"):
    images_dic = {"id":[], "data": [], "labels": []}
    for i in range(20):
        if type == "train":
            start, end = data_split_dic[i]["train_range"]
        else:
            start, end = data_split_dic[i]["test_range"]
        for j in range(start, end + 1):
            image = io.imread(f"data/origin/{j}.jpg")
            images_dic["id"].append(j)
            images_dic["data"].append(image)
            images_dic["labels"].append(i)
    return images_dic

train_data = get_images("train")
test_data = get_images("test")

In [ ]:
def extract_graycomatrix_features(image):
    gray_image = color.rgb2gray(image)
    gray_image = (gray_image * 255).astype('uint8')
    distances = [1,3,5]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    
    glcm = feature.graycomatrix(gray_image, distances, angles, symmetric=True, normed=True)

    contrast = feature.graycoprops(glcm, 'contrast')
    dissimilarity = feature.graycoprops(glcm, 'dissimilarity')
    homogeneity = feature.graycoprops(glcm, 'homogeneity')
    energy = feature.graycoprops(glcm, 'energy')
    correlation = feature.graycoprops(glcm, 'correlation')
    asm = feature.graycoprops(glcm, 'ASM')

    features = torch.tensor([
        np.mean(contrast),
        np.mean(dissimilarity),
        np.mean(homogeneity),
        np.mean(energy),
        np.mean(correlation),
        np.mean(asm)
    ], dtype=torch.float32)
    
    return features

In [ ]:
def extract_sift_descriptors(image):
    gray_image = color.rgb2gray(image)
    gray_image_uint8 = (gray_image * 255).astype('uint8')
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image_uint8, None)
    return descriptors

def build_sift_vocabulary(data, vocabulary_size=200):
    descriptors_list = []

    for image in tqdm.tqdm(data["data"], desc="Extracting SIFT descriptors"):
        descriptors = extract_sift_descriptors(image)
        if descriptors is not None and len(descriptors) > 0:
            descriptors_list.append(descriptors)

    descriptors_np = np.vstack(descriptors_list)
    kmeans = MiniBatchKMeans(n_clusters=vocabulary_size, random_state=42, batch_size=vocabulary_size*5, n_init='auto')
    kmeans.fit(descriptors_np)
    return kmeans

def extract_sift_bow_features(image, kmeans_model, vocabulary_size=200):
    descriptors = extract_sift_descriptors(image)
    if descriptors is None or len(descriptors) == 0:
        return torch.zeros(vocabulary_size, dtype=torch.float32)
    
    labels = kmeans_model.predict(descriptors)
    hist = np.bincount(labels, minlength=vocabulary_size)
    
    if np.sum(hist) > 0:
        hist = hist.astype(np.float32) / np.sum(hist)
    else:
        hist = hist.astype(np.float32)
    
    return torch.tensor(hist, dtype=torch.float32)

In [ ]:
def extract_color_histogram_features(image, bins_per_channel=32):
    histogram_r = exposure.histogram(image[:, :, 0], nbins=bins_per_channel, source_range='dtype')
    histogram_g = exposure.histogram(image[:, :, 1], nbins=bins_per_channel, source_range='dtype')
    histogram_b = exposure.histogram(image[:, :, 2], nbins=bins_per_channel, source_range='dtype')

    histogram = np.concatenate((histogram_r[0], histogram_g[0], histogram_b[0]))

    return torch.tensor(histogram, dtype=torch.float32)

In [ ]:
def get_features_for_dataset(data, kmeans_model, vocabulary_size=200):
    dataset = {
        "id": [],
        "image":[],
        "bow": [],
        "graycomatrix":[],
        "color_histogram": [],
        "labels": [],
        }
    
    for id, image, label in tqdm.tqdm(zip(data["id"], data["data"], data["labels"]), 
                                  desc="Extracting features",
                                  total=len(data["data"])):
        bow_features = extract_sift_bow_features(image, kmeans_model, vocabulary_size)
        graycomatrix_features = extract_graycomatrix_features(image)
        color_histogram_features = extract_color_histogram_features(image)
        dataset["id"].append(id)
        dataset["image"].append(image)
        dataset["bow"].append(bow_features)
        dataset["graycomatrix"].append(graycomatrix_features)
        dataset["color_histogram"].append(color_histogram_features)
        dataset["labels"].append(label)
    
    return dataset

In [ ]:
kmeans_model = build_sift_vocabulary(train_data, vocabulary_size=200)

train_dataset = get_features_for_dataset(train_data, kmeans_model, vocabulary_size=200)
test_dataset = get_features_for_dataset(test_data, kmeans_model, vocabulary_size=200)